In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.drop = nn.Dropout(0.1)
        self.conv0_0 = nn.Conv2d(1, 32, 3, padding=1) #input 1x28x28 OUtput32x28x28 RF 3x3
        self.bn0_0 = nn.BatchNorm2d(32)
        self.conv0_1 = nn.Conv2d(32,16 , 3, padding=1)#input 32x28x28 OUtput 64x28x28 RF 5x5
        self.bn0_1 = nn.BatchNorm2d(16)
        self.pool0 = nn.MaxPool2d(2, 2)#input 64x28x28 OUtput 64x14x14 RF 10x10
        self.conv1_0 = nn.Conv2d(16, 16, 3, padding=1)
        self.bn1_0 = nn.BatchNorm2d(16)
        self.conv1_1 = nn.Conv2d(16, 16, 3, padding=1)#input 64x14x14 OUtput 128x14x14 RF 12x12
        self.bn1_1 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2_0 = nn.Conv2d(16, 16, 3, padding=1)#input 128x14x14 OUtput 256x14x14 RF 14x14
        self.bn2_0 = nn.BatchNorm2d(16)
        self.conv2_1 = nn.Conv2d(16, 16, 3, padding=1)#input 128x14x14 OUtput 256x14x14 RF 14x14
        self.bn2_1 = nn.BatchNorm2d(16)
        self.pool2 = nn.MaxPool2d(2, 2)#input -256x14x14 OUtput256x7x7 RF 28x28
        self.conv3_0 = nn.Conv2d(16, 16, 3, padding=1)
        self.bn3_0 = nn.BatchNorm2d(16)
        self.conv3_1 = nn.Conv2d(16, 16, 3, padding=1)
        self.fc = nn.Linear(16,10)
    def forward(self, x):
        x = self.pool0(self.drop(F.relu(self.bn0_1(self.conv0_1(self.drop(F.relu(self.bn0_0(self.conv0_0(x)))))))))
        #print(x.shape)
        x = self.pool1(self.drop(F.relu(self.bn1_1(self.conv1_1(self.drop(F.relu(self.bn1_0(self.conv1_0(x)))))))))
        #print(x.shape)
        x = self.pool1(self.drop(F.relu(self.bn2_1(self.conv2_1(self.drop(F.relu(self.bn2_0(self.conv2_0(x)))))))))
        #print(x.shape)
        x = F.adaptive_avg_pool2d(self.conv3_1(self.drop(F.relu(self.bn3_0(self.conv3_0(x))))), output_size=1)
        #print(x.shape)
        x = self.fc(x.view(-1, 16))
        return F.log_softmax(x)

In [ ]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
       BatchNorm2d-2           [-1, 32, 28, 28]              64
           Dropout-3           [-1, 32, 28, 28]               0
            Conv2d-4           [-1, 16, 28, 28]           4,624
       BatchNorm2d-5           [-1, 16, 28, 28]              32
           Dropout-6           [-1, 16, 28, 28]               0
         MaxPool2d-7           [-1, 16, 14, 14]               0
            Conv2d-8           [-1, 16, 14, 14]           2,320
       BatchNorm2d-9           [-1, 16, 14, 14]              32
          Dropout-10           [-1, 16, 14, 14]               0
           Conv2d-11           [-1, 16, 14, 14]           2,320
      BatchNorm2d-12           [-1, 16, 14, 14]              32
    

<ipython-input-79-a788e45283a9>:34: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [ ]:

torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-79-a788e45283a9>:34: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.06989356130361557 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 30.10it/s]



Test set: Average loss: 0.0713, Accuracy: 9806/10000 (98.06%)



loss=0.09881863743066788 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 29.80it/s]



Test set: Average loss: 0.0490, Accuracy: 9847/10000 (98.47%)



loss=0.054680854082107544 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 29.48it/s]


...


Test set: Average loss: 0.0234, Accuracy: 9928/10000 (99.28%)



loss=0.005360124632716179 batch_id=468: 100%|██████████| 469/469 [00:17<00:00, 27.49it/s]



Test set: Average loss: 0.0233, Accuracy: 9928/10000 (99.28%)



loss=0.006331129465252161 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 28.50it/s]



Test set: Average loss: 0.0258, Accuracy: 9925/10000 (99.25%)

